In [ ]:
import polars as pl

In [ ]:
t_historico = pl.read_csv("..\GESTION DE ATENCIONES\Tickets\Tickets Historico.txt", separator=";")
t_historico = t_historico.select(
    pl.col('Numero Ticket').alias('TicketID'),
    'Ubicacion',
    'Service Desk',
    'Estado',
    pl.col('Fecha Creacion').str.to_date(),
    pl.col('Fecha Termino').str.to_date(),
    pl.col('Fecha Cierre').str.to_date()
)

In [ ]:
t_actual = pl.read_csv("..\GESTION DE ATENCIONES\Tickets\Tickets Actual.csv",separator="|")
t_actual = t_actual.select(
    pl.col('Numero Ticket').alias('TicketID'),
    'Ubicacion',
    'Service Desk',
    'Estado',
    pl.col('Fecha Creacion').str.to_date(),
    pl.col('Fecha Termino').str.to_date(),
    pl.col('Fecha Cierre').str.to_date()
).filter(pl.col('TicketID').str.starts_with("WO"))

In [ ]:
#Consolidadndo datasets Historico y actual
tickets = pl.concat([t_historico, t_actual])

In [ ]:
#Dividir columna Ubicacion

tickets = tickets.with_columns(
    pl.col("Ubicacion")
    .str.split_exact('-',1)
    .struct.rename_fields(['Agencia','AgenciaID'])  
    .alias('Agencia') 
).unnest('Agencia')


In [ ]:
tickets.head()

In [ ]:
#Castear tipo de dato entero a la columna AgenciaID
##tickets = tickets.cast({'AgenciaID': pl.Int64})

In [32]:
#Eliminando duplicados
tickets.sort(
    by=['TicketID','Fecha Creacion']
).unique(
    subset='TicketID',
    keep='last',
    maintain_order=True
).shape

(22215, 9)

In [33]:
#Creando columna Fecha Real Fint
tickets = tickets.with_columns(
    pl.when(pl.col('Fecha Termino').is_null())
    .then(pl.col('Fecha Cierre'))
    .otherwise(pl.col('Fecha Termino'))
    .alias('Fecha Real Fin')
)

In [36]:
#tickets.filter(pl.col('Fecha Termino').is_null())

TicketID,Ubicacion,Service Desk,Estado,Fecha Creacion,Fecha Termino,Fecha Cierre,Agencia,AgenciaID,Fecha Real Fin
str,str,str,str,date,date,date,str,str,date


In [37]:
#Creando columna Dias Cierre
tickets = tickets.with_columns(
    (pl.col('Fecha Real Fin') - pl.col('Fecha Creacion')).dt.total_days().alias('Dias Cierre')
)

TicketID,Ubicacion,Service Desk,Estado,Fecha Creacion,Fecha Termino,Fecha Cierre,Agencia,AgenciaID,Fecha Real Fin,Dias Cierre
str,str,str,str,date,date,date,str,str,date,i64
"""WO000000412268…","""AREQUIPA - 215…","""Zona Norte""","""Cerrado""",2022-06-01,2022-06-14,2022-06-14,"""AREQUIPA """,""" 215000""",2022-06-14,13
"""WO000000412264…","""METRO AV. WIES…","""Zona Centro""","""Cerrado""",2022-06-01,2022-06-17,2022-06-17,"""METRO AV. WIES…",""" 191106""",2022-06-17,16
"""WO000000412250…","""NICOLAS AYLLON…","""Zona Centro""","""Cerrado""",2022-06-01,2022-06-08,2022-06-08,"""NICOLAS AYLLON…",""" 191027""",2022-06-08,7
"""WO000000412251…","""JAUREGUI - 405…","""Zona Norte""","""Cerrado""",2022-06-01,2022-06-18,2022-06-18,"""JAUREGUI """,""" 405005""",2022-06-18,17
"""WO000000412274…","""CANTO GRANDE -…","""Zona Centro""","""Cerrado""",2022-06-01,2022-06-08,2022-06-08,"""CANTO GRANDE """,""" 191096""",2022-06-08,7
